## Plots delays, throughputs, packet loss and path for a selected link

In [4]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
indices = "network_weather-2017.5.*"

my_query = {}

from sklearn import tree

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import roc_curve, auc

import time
from sklearn.model_selection import train_test_split

import random

from pandas.tseries.offsets import *

#from graphviz import Source

ImportError: No module named 'elasticsearch'

##  Select your link

In [ ]:
# sS='UC'
# srcSiteOWDServer = "192.170.227.160"
# srcSiteThroughputServer = "192.170.227.162"

sS1= 'pic'
srcSiteOWDServer1 = "193.109.172.188"
#srcSiteThroughputServer = "128.142.223.246"

sS2= 'pic'
srcSiteOWDServer2 = "193.109.172.188"

sS3= 'pic'
srcSiteOWDServer3 = "193.109.172.188"

sS4= 'pic'
srcSiteOWDServer4 = "193.109.172.188"

sS5= 'pic'
srcSiteOWDServer5 = "193.109.172.188"

# dS='IU'
# destSiteOWDServer = "149.165.225.223"
# destSiteThroughputServer = "149.165.225.224"

# dS='UIUC'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.96.9"

# dS='ICCN'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.126.132"

dS1 = 'CERN-PROD'
destSiteOWDServer1 = "128.142.223.247"
#destSiteThroughputServer = "193.109.172.187"

dS2 = 'RO-16-UAIC'
destSiteOWDServer2 = "85.122.31.74"

dS3 = 'praguelcg2'
destSiteOWDServer3 = "147.231.25.192"

dS4 = 'MWT2'
destSiteOWDServer4 = "149.165.225.223"

dS5 = 'UFlorida-HPC'
destSiteOWDServer5 = "128.227.221.44"

## Get the data from Elasticsearch, and store the record based on its type

In [ ]:
my_query1 = {
    'query': { 
       'bool':{
            'must':[
                    {'range': {'timestamp': {'gte': '20170510T000000Z', 'lt': '20170517T000000Z'}}},
                        {'bool':
                            {'should':[
                                {'term': {'src': srcSiteOWDServer1}},
                            ]}
                        },
                        {'bool':
                            {'should':[
                                {'term': {'dest': destSiteOWDServer1}},
                            ]}
                        },
                        {'bool':
                            {'should':[
                                {'term': {'_type': 'packet_loss_rate'}},
                            ]}
                        }
                    ]
                }
        
        }
    }


scroll1 = list(scan(client=es, index=indices, query=my_query1))

In [ ]:
my_query2 = {
    'query': { 
       'bool':{
            'must':[
                    {'range': {'timestamp': {'gte': '20170510T000000Z', 'lt': '20170517T000000Z'}}},
                        {'bool':
                            {'should':[
                                {'term': {'src': srcSiteOWDServer2}},
                            ]}
                        },
                        {'bool':
                            {'should':[
                                {'term': {'dest': destSiteOWDServer2}},
                            ]}
                        },
                        {'bool':
                            {'should':[
                                {'term': {'_type': 'packet_loss_rate'}},
                            ]}
                        }
                    ]
                }
        
        }
    }


scroll2 = list(scan(client=es, index=indices, query=my_query2))

In [ ]:
my_query3 = {
    'query': { 
       'bool':{
            'must':[
                    {'range': {'timestamp': {'gte': '20170510T000000Z', 'lt': '20170517T000000Z'}}},
                        {'bool':
                            {'should':[
                                {'term': {'src': srcSiteOWDServer3}},
                            ]}
                        },
                        {'bool':
                            {'should':[
                                {'term': {'dest': destSiteOWDServer3}},
                            ]}
                        },
                        {'bool':
                            {'should':[
                                {'term': {'_type': 'packet_loss_rate'}},
                            ]}
                        }
                    ]
                }
        
        }
    }


scroll3 = list(scan(client=es, index=indices, query=my_query3))

In [ ]:
my_query4 = {
    'query': { 
       'bool':{
            'must':[
                    {'range': {'timestamp': {'gte': '20170510T000000Z', 'lt': '20170517T000000Z'}}},
                        {'bool':
                            {'should':[
                                {'term': {'src': srcSiteOWDServer4}},
                            ]}
                        },
                        {'bool':
                            {'should':[
                                {'term': {'dest': destSiteOWDServer4}},
                            ]}
                        },
                        {'bool':
                            {'should':[
                                {'term': {'_type': 'packet_loss_rate'}},
                            ]}
                        }
                    ]
                }
        
        }
    }


scroll4 = list(scan(client=es, index=indices, query=my_query4))

In [ ]:
my_query5 = {
    'query': { 
       'bool':{
            'must':[
                    {'range': {'timestamp': {'gte': '20170510T000000Z', 'lt': '20170517T000000Z'}}},
                        {'bool':
                            {'should':[
                                {'term': {'src': srcSiteOWDServer5}},
                            ]}
                        },
                        {'bool':
                            {'should':[
                                {'term': {'dest': destSiteOWDServer5}},
                            ]}
                        },
                        {'bool':
                            {'should':[
                                {'term': {'_type': 'packet_loss_rate'}},
                            ]}
                        }
                    ]
                }
        
        }
    }


scroll5 = list(scan(client=es, index=indices, query=my_query5))

In [ ]:
site1 = {srcSiteOWDServer1: sS1, destSiteOWDServer1:dS1}
data1 = {sS1:pd.DataFrame(),dS1:pd.DataFrame()}
epoch = datetime.utcfromtimestamp(0)
count = 0
print(epoch)

for res in scroll1:
    if count<10: print(res)
    count += 1
#     dt=datetime.strptime(res['_source']['timestamp'], '%Y-%m-%dT%H:%M:%S') # for data before Oct. 15 2016
    dt=datetime.utcfromtimestamp(res['_source']['timestamp']/1000.0)
    ts = (dt - epoch).total_seconds() * 1000.0
    s=site1[res['_source']['src']]
    data1[s].set_value(ts, 'timestamp', dt)
    column_type = res['_type']
    if column_type == 'packet_loss_rate':
        data1[s].set_value( ts, 'packet_loss', res['_source']['packet_loss'])
    #if count<2: print(dt)

In [ ]:
site2 = {srcSiteOWDServer2: sS2, destSiteOWDServer2:dS2}
data2 = {sS2:pd.DataFrame(),dS2:pd.DataFrame()}
epoch = datetime.utcfromtimestamp(0)
count = 0
print(epoch)

for res in scroll2:
    if count<10: print(res)
    count += 1
#     dt=datetime.strptime(res['_source']['timestamp'], '%Y-%m-%dT%H:%M:%S') # for data before Oct. 15 2016
    dt=datetime.utcfromtimestamp(res['_source']['timestamp']/1000.0)
    ts = (dt - epoch).total_seconds() * 1000.0
    s=site2[res['_source']['src']]
    data2[s].set_value(ts, 'timestamp', dt)
    column_type = res['_type']
    if column_type == 'packet_loss_rate':
        data2[s].set_value( ts, 'packet_loss', res['_source']['packet_loss'])
    #if count<2: print(dt)

In [ ]:
site3 = {srcSiteOWDServer3: sS3, destSiteOWDServer3:dS3}
data3 = {sS3:pd.DataFrame(),dS3:pd.DataFrame()}
epoch = datetime.utcfromtimestamp(0)
count = 0
print(epoch)

for res in scroll3:
    if count<10: print(res)
    count += 1
#     dt=datetime.strptime(res['_source']['timestamp'], '%Y-%m-%dT%H:%M:%S') # for data before Oct. 15 2016
    dt=datetime.utcfromtimestamp(res['_source']['timestamp']/1000.0)
    ts = (dt - epoch).total_seconds() * 1000.0
    s=site3[res['_source']['src']]
    data3[s].set_value(ts, 'timestamp', dt)
    column_type = res['_type']
    if column_type == 'packet_loss_rate':
        data3[s].set_value( ts, 'packet_loss', res['_source']['packet_loss'])
    #if count<2: print(dt)

In [ ]:
site4 = {srcSiteOWDServer4: sS4, destSiteOWDServer4:dS4}
data4 = {sS4:pd.DataFrame(),dS4:pd.DataFrame()}
epoch = datetime.utcfromtimestamp(0)
count = 0
print(epoch)

for res in scroll4:
    if count<10: print(res)
    count += 1
#     dt=datetime.strptime(res['_source']['timestamp'], '%Y-%m-%dT%H:%M:%S') # for data before Oct. 15 2016
    dt=datetime.utcfromtimestamp(res['_source']['timestamp']/1000.0)
    ts = (dt - epoch).total_seconds() * 1000.0
    s=site4[res['_source']['src']]
    data4[s].set_value(ts, 'timestamp', dt)
    column_type = res['_type']
    if column_type == 'packet_loss_rate':
        data4[s].set_value( ts, 'packet_loss', res['_source']['packet_loss'])
    #if count<2: print(dt)

In [ ]:
site5 = {srcSiteOWDServer5: sS5, destSiteOWDServer5:dS5}
data5 = {sS5:pd.DataFrame(),dS5:pd.DataFrame()}
epoch = datetime.utcfromtimestamp(0)
count = 0
print(epoch)

for res in scroll5:
    if count<10: print(res)
    count += 1
#     dt=datetime.strptime(res['_source']['timestamp'], '%Y-%m-%dT%H:%M:%S') # for data before Oct. 15 2016
    dt=datetime.utcfromtimestamp(res['_source']['timestamp']/1000.0)
    ts = (dt - epoch).total_seconds() * 1000.0
    s=site5[res['_source']['src']]
    data5[s].set_value(ts, 'timestamp', dt)
    column_type = res['_type']
    if column_type == 'packet_loss_rate':
        data5[s].set_value( ts, 'packet_loss', res['_source']['packet_loss'])
    #if count<2: print(dt)

In [ ]:
# print("hi")
# exit()

In [ ]:
data1[sS1].sort_index(inplace=True) 
data1[sS1].describe()

print(data1[sS1])

In [ ]:
data2[sS2].sort_index(inplace=True) 
data2[sS2].describe()

print(data2[sS2])

In [ ]:
data3[sS3].sort_index(inplace=True) 
data3[sS3].describe()

print(data3[sS3])

In [ ]:
data4[sS4].sort_index(inplace=True) 
data4[sS4].describe()

print(data4[sS4])

In [ ]:
data5[sS5].sort_index(inplace=True) 
data5[sS5].describe()

print(data5[sS5])

### Extract values

In [ ]:
F1 = data1[sS1]
Fpl1 = F1[F1.packet_loss.notnull()]

#print(Fpl)
print(Fpl1)

In [ ]:
F2 = data2[sS2]
Fpl2 = F2[F2.packet_loss.notnull()]

#print(Fpl)
print(Fpl2)

In [ ]:
F3 = data3[sS3]
Fpl3 = F3[F3.packet_loss.notnull()]

#print(Fpl)
print(Fpl3)

In [ ]:
F4 = data4[sS4]
Fpl4 = F4[F4.packet_loss.notnull()]

#print(Fpl)
print(Fpl4)

In [ ]:
F5 = data5[sS5]
Fpl5 = F5[F5.packet_loss.notnull()]

#print(Fpl)
print(Fpl5)

#### interpolate all the values and leave only onese where all 4 measurements are there

In [ ]:
# NF=F.interpolate(method='index',limit=1,limit_direction='both')
# NFall = NF[NF.packet_loss.notnull()]

# print(NFall.shape)

In [ ]:
# print(NF.shape)

In [ ]:
print( 'packet loss forward: ', Fpl1.shape[0])

In [ ]:
print( 'packet loss forward: ', Fpl2.shape[0])

In [ ]:
print( 'packet loss forward: ', Fpl3.shape[0])

In [ ]:
print( 'packet loss forward: ', Fpl4.shape[0])

In [ ]:
print( 'packet loss forward: ', Fpl5.shape[0])

In [ ]:
plt.figure(figsize=[16, 17])
gs = gridspec.GridSpec(4, 1)

ax0 = plt.subplot(gs[1])
ax0.plot(Fpl1.timestamp.tolist(), np.sqrt(Fpl1.packet_loss).tolist(), ls='', marker='.', c='r', label=sS+'->'+dS)
ax0.set_xlabel('time')
ax0.set_ylabel('sqrt(packet loss) [%]')
ax0.legend()

ax2 = plt.subplot(gs[1])
ax2.plot(Fpl2.timestamp.tolist(), np.sqrt(Fpl2.packet_loss).tolist(), ls='', marker='.', c='r', label=sS+'->'+dS)
ax2.set_xlabel('time')
ax2.set_ylabel('sqrt(packet loss) [%]')
ax2.legend()

ax4 = plt.subplot(gs[1])
ax4.plot(Fpl4.timestamp.tolist(), np.sqrt(Fpl4.packet_loss).tolist(), ls='', marker='.', c='r', label=sS+'->'+dS)
ax4.set_xlabel('time')
ax4.set_ylabel('sqrt(packet loss) [%]')
ax4.legend()

ax6 = plt.subplot(gs[1])
ax6.plot(Fpl6.timestamp.tolist(), np.sqrt(Fpl6.packet_loss).tolist(), ls='', marker='.', c='r', label=sS+'->'+dS)
ax6.set_xlabel('time')
ax6.set_ylabel('sqrt(packet loss) [%]')
ax6.legend()

ax8 = plt.subplot(gs[1])
ax8.plot(Fpl8.timestamp.tolist(), np.sqrt(Fpl8.packet_loss).tolist(), ls='', marker='.', c='r', label=sS+'->'+dS)
ax8.set_xlabel('time')
ax8.set_ylabel('sqrt(packet loss) [%]')
ax8.legend()

plt.show()

### Combining the dataframes

In [ ]:
Fpl1.set_index('timestamp').join(Fpl2.set_index('timestamp'))

In [ ]:
print(Fpl1)

In [ ]:
print("Hi")
exit()

### Boosted Decision Trees

In [ ]:
start_time = time.clock()

# predicted_flag_array = np.zeros((24*3600), dtype=np.int)
# auc_score_array = np.zeros((24*3600), dtype=np.float)
# score_holder_array = np.zeros(3600, dtype=np.float)
# anomaly_array = np.ones(3600, dtype=np.int)
# not_anomaly_array = np.zeros(3600, dtype=np.int)

In [ ]:
n_series = 1
start_date = '2017-08-14 00:00:00'
end_date = '2017-08-21 23:59:59'

window = 24

In [ ]:
def check_for_anomaly(ref, sub):
    
    y_ref = pd.Series([0] * ref.shape[0])
    X_ref = ref
    del X_ref['auc_score']
    
    y_sub = pd.Series([1] * sub.shape[0])
    X_sub=sub
    del X_sub['auc_score']
    
    # separate Reference and Subject into Train and Test
    X_ref_train, X_ref_test, y_ref_train, y_ref_test = train_test_split(X_ref, y_ref, test_size=0.3, random_state=50)
    X_sub_train, X_sub_test, y_sub_train, y_sub_test = train_test_split(X_sub, y_sub, test_size=0.3, random_state=50)
    
    # combine training ref and sub samples
    X_train = pd.concat([X_ref_train, X_sub_train])
    y_train = pd.concat([y_ref_train, y_sub_train])

    # combine testing ref and sub samples
    X_test = pd.concat([X_ref_test, X_sub_test])
    y_test = pd.concat([y_ref_test, y_sub_test])
    
    clf = AdaBoostClassifier(n_estimators=50)
    
    #train an AdaBoost model to be able to tell the difference between the reference and subject data
    clf.fit(X_train, y_train) 

    #Predict using the combined test data
    y_predict = clf.predict(X_test)
    
    # scores = cross_val_score(clf, X, y)
    # print(scores)
    
    fpr, tpr, thresholds = roc_curve(y_test, y_predict) # calculate the false positive rate and true positive rate
    auc_score = auc(fpr, tpr) #calculate the AUC score
    print ( "auc_score = ", auc_score, "\tfeature importances:", clf.feature_importances_)
    
    if auc_score > 0.55: 
        plot_roc(fpr, tpr, auc_score)
        #filename='tree_'+sub.index.min().strftime("%Y-%m-%d_%H")
        #tree.export_graphviz(clf.estimators_[0] , out_file=filename +'_1.dot') 
        #tree.export_graphviz(clf.estimators_[1] , out_file=filename +'_2.dot') 
        
    return auc_score

In [ ]:
def plot_roc(fpr,tpr, roc_auc):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.plot([0, 1], [0, 1], linestyle='--', color='r',label='Luck', alpha=.8)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
start_date = Fpl['timestamp'].min()
print(start_date)
end_date = Fpl['timestamp'].max()
print(end_date)

#print(Fpl.index/1000 - 1502668800.0)

Fpl2 = Fpl.set_index('timestamp')
Fpl2['auc_score']=0.5
print(Fpl2)

In [ ]:
print(Fpl2.index)

In [ ]:
#find min and max timestamps

start = Fpl2.index.min()
end = Fpl2.index.max()

#round start 
start.seconds=0
start.minutes=0

ref = window * Hour()
sub = 1 * Hour()

# loop over them
ti=start+ref+sub
count=0
while ti < end + 1 * Minute():
    ref_start = ti-ref-sub
    ref_end = ti-sub
    ref_df = Fpl2[(Fpl2.index >= ref_start) & (Fpl2.index < ref_end)]
    #print('ref_df = ', ref_df)
    #print("In while loop: ref_df: ", ref_df)
    sub_df = Fpl2[(Fpl2.index >= ref_end) & (Fpl2.index < ti)]
    #print('sub_df = ', sub_df)
    #print("In while loop: sub_df: ", sub_df)
    
    ref_count = ref_df.shape[0]
    sub_count = sub_df.shape[0] 
    if ref_count < 10 or sub_count < 3:
        auc_score == 0.5
        ti = ti + sub
        count=count+1
        continue
    
    print("timestamp = ", ref_end)
    auc_score = check_for_anomaly(ref_df, sub_df)
    Fpl2.loc[(Fpl2.index>=ref_end) & (Fpl2.index<=ti),['auc_score']] = auc_score
    #print(ti,"\trefes:" , ref_df.shape[0], "\tsubjects:", sub_df.shape[0], '\tauc:', auc_score)
    ti = ti + sub
    count=count+1
    


In [ ]:
Fpl2.plot(figsize=(20,7))


In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
Fpl2.loc[:,'Detected'] = 0
Fpl2.loc[Fpl2.auc_score>0.55,'Detected']=1
Fpl2.head()
#ax.plot(Fpl2.Flag, 'r')
ax.plot(Fpl2.auc_score,'g')
ax.fill( Fpl2.Detected, 'b', alpha=0.3)
ax.plot(Fpl.timestamp.tolist(), np.sqrt(Fpl.packet_loss).tolist(), ls='', linestyle='-', marker='.', c='r', label=sS+'->'+dS)
ax.set_xlabel('time')
ax.set_ylabel('sqrt(packet loss) [%]')
ax.legend(loc='upper left')
plt.show()

In [ ]:
print(Fpl2)

In [ ]:
#Fpl2_avg_auc_holder = Fpl2.auc_score
#Fpl2_avg_auc = Fpl2_avg_auc_holder[:'2017-08-15 00:00:00']
#Fpl2_avg_auc.mean()
#ref_start_df = pd.DataFrame(columns=['timestamp'])

#columns1 = ['timestamp', 'auc_score']
#ref_start_df = pd.DataFrame(columns=columns1) 
#ref_start_df.set_index('timestamp')

#print(ref_start_df)




In [ ]:
Fpl2_avg_pkt_holder = Fpl2.packet_loss
#holds all the auc scores

timestamp_np = np.array([])
pkt_score_np = np.array([])

start = Fpl2.index.min()
end = Fpl2.index.max()

#round start 
start.seconds=0
start.minutes=0

ref = window * Hour()
sub = 1 * Hour()

# loop over them
ti=start
count=0
while ti < end + 1 * Minute():
#while start < end:
    ref_start = ti
    ref_end = ti+sub
    
    
    timestamp_np = np.append(timestamp_np, ref_start)
    #create a dataframe that stores the starting time of the each hour
    
    Fpl2_avg_pkt = math.sqrt(Fpl2_avg_pkt_holder[ref_start:ref_end].mean())
    #average packet loss score for every hour    
    
    pkt_score_np = np.append(pkt_score_np, Fpl2_avg_pkt)
    
    #start = start + sub
    
    ti = ti + sub
    #count=count+1
    #if count>2: break

timestamp1_df = pd.DataFrame(timestamp_np[:], columns=['timestamp'])
pkt_score1_df = pd.DataFrame(pkt_score_np[:], columns=['avg_packet_loss'])

pkt_avg_df1 = pd.concat([timestamp1_df, pkt_score1_df], axis=1)

pkt_avg_df2 = pkt_avg_df1.set_index('timestamp')

print(pkt_avg_df2)


In [ ]:
pkt_avg_df2.plot(figsize=(20,7))

In [ ]:
# ax = pkt_avg_df2.plot()
# Fpl2.loc[:,'Detected'] = 0
# Fpl2.loc[Fpl2.auc_score>0.51,'Detected']=1
# Fpl2.plot(ax=ax, figsize=(20,7))
# ax.fill( Fpl2.Detected, 'b', alpha=0.3)

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
Fpl2.loc[:,'Detected'] = 0
Fpl2.loc[Fpl2.auc_score>0.55,'Detected']=1
Fpl2.head()
#ax.plot(Fpl2.Flag, 'r')
ax.plot(Fpl2.auc_score,'g')
ax.fill( Fpl2.Detected, 'b', alpha=0.3)
ax.plot(timestamp1_df, pkt_avg_df2, ls='', linestyle='-', marker='.', c='r', label=sS+'->'+dS)
ax.set_xlabel('time')
ax.set_ylabel('sqrt(packet loss) [%]')
ax.legend(loc='upper left')
plt.show()